In [2]:
import pandas as pd
import json

In [3]:
# with open('yelp_dataset/yelp_academic_dataset_checkin.json', 'r', encoding='utf-8') as file:
#     data_checkin = [json.loads(line) for line in file]

# df_checkin = pd.DataFrame(data_checkin)
# df_checkin.head()

In [4]:
# with open('yelp_dataset/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as file:
#     data_business = [json.loads(line) for line in file]

# df_business = pd.DataFrame(data_business)
# df_business.head()

In [5]:
# with open('yelp_dataset/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as file:
#     data_review = [json.loads(line) for line in file]

# df_review = pd.DataFrame(data_review)
# df_review.head()

In [6]:
# with open('yelp_dataset/yelp_academic_dataset_tip.json', 'r', encoding='utf-8') as file:
#     data_tip = [json.loads(line) for line in file]

# df_tip = pd.DataFrame(data_tip)
# df_tip.head()

In [7]:
# with open('yelp_dataset/yelp_academic_dataset_user.json', 'r', encoding='utf-8') as file:
#     data_user = [json.loads(line) for line in file]

# df_user = pd.DataFrame(data_user)
# df_user.head()

In [8]:
import pymongo

# Connect to the MongoDB server running on localhost (default port 27017)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access a specific database (replace 'your_database' with your database name)
db = client['yelp']


In [9]:
# Access a specific collection within the database (replace 'your_collection' with your collection name)
collection_business = db['business']
collection_review = db['review']


In [10]:
# Find documents that match a specific condition
query = {'name': "The UPS Store"}
result = collection_business.find(query)

# Iterate through the result
for doc in result:
    print(doc)


{'_id': ObjectId('656b29078895f3b51cb6cc69'), 'business_id': 'mpf3x-BjTdTEA3yCZrAYPw', 'name': 'The UPS Store', 'address': '87 Grasso Plaza Shopping Center', 'city': 'Affton', 'state': 'MO', 'postal_code': '63123', 'latitude': 38.551126, 'longitude': -90.335695, 'stars': 3, 'review_count': 15, 'is_open': 1, 'attributes': {'BusinessAcceptsCreditCards': 'True'}, 'categories': 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services', 'hours': {'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}}
{'_id': ObjectId('656b29078895f3b51cb6cd55'), 'business_id': 'TF6QZYUR5atqBEFnHP5bOw', 'name': 'The UPS Store', 'address': '748 S Meadows Pkwy, Ste A-9', 'city': 'Reno', 'state': 'NV', 'postal_code': '89521', 'latitude': 39.4369354248, 'longitude': -119.7573776245, 'stars': 3.5, 'review_count': 27, 'is_open': 1, 'attributes': {'BusinessAcceptsCreditCards': 'True'}, 'categories': 'Loca

In [11]:
user_input = input("Enter the business name: ")

# Access the collections
business_collection = db['business']
review_collection = db['review']

# Find in business collection where name matches user input
business_query = {"name": user_input}
business_result = business_collection.find_one(business_query)

# Check if business found in the business collection
if business_result:
    # If business found, find in review collection where stars = 1 and business_id matches
    review_query = {"stars": {'$in': [1, 2]}, "business_id": business_result["business_id"]}  # Assuming business_id field is used for matching
    review_result = review_collection.find(review_query)

    # Print review documents
    for review in review_result:
        print(review)
else:
    print("No business found with name 'The UPS Store' in the business collection.")


{'_id': ObjectId('656b23948895f3b51c4e5643'), 'review_id': 'ptb-7Yy03TwRRXLCCMaSNA', 'user_id': 'LhHOF4H2vz9xrcfJn3pCZA', 'business_id': 'tUFrWirKiKi_TAnsVWINQQ', 'stars': 2, 'useful': 3, 'funny': 0, 'cool': 0, 'text': "This Target misses the mark. \n\nTarget is Target.... but this one was squeezed into an existing space and the layout is completely discombobulated. \n\nIf you don't mind searching and searching you'll like this place otherwise set your sights on somewhere else.", 'date': '2008-12-14 08:41:21'}
{'_id': ObjectId('656b23a08895f3b51c4fef9c'), 'review_id': '8mpUJc1U4lpDeKUxVOnrTQ', 'user_id': 'JUT0U3HTSB3kz9Wh7N0GqA', 'business_id': 'tUFrWirKiKi_TAnsVWINQQ', 'stars': 2, 'useful': 8, 'funny': 11, 'cool': 2, 'text': 'I\'m going to get real with you for a sec Target, and this may hurt... how many Target\'s on Broadway do we need? This one feels like an afterthought, not a real Target, just a slight suggestion of what a Target should actually be. The layout drives me nuts, it\'

In [12]:
import pymongo
from datetime import datetime
import uuid

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the database
db = client['yelp']  # Replace 'your_database' with your actual database name

# Access the collections
review_collection = db['review']
user_collection = db['user']
business_collection = db['business']

def add_review(stars, useful, funny, cool, text, business_name, user_name):
    # Generate new random IDs for review and user
    review_id = str(uuid.uuid4())
    user_id = str(uuid.uuid4())

    # Get current date and time
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Find business_id based on business_name
    business = business_collection.find_one({"name": business_name})
    if not business:
        return "Business not found"
    business_id = business['business_id']

    # Find user_id based on user_name
    user = user_collection.find_one({"name": user_name})
    if not user:
        return "User not found"
    user_id = user['user_id']

    # Construct the review document
    review_doc = {
        "_id": review_id,
        "review_id": str(uuid.uuid4()),
        "user_id": user_id,
        "business_id": business_id,
        "stars": stars,
        "useful": useful,
        "funny": funny,
        "cool": cool,
        "text": text,
        "date": current_date
    }

    # Insert the review document into the review collection
    review_collection.insert_one(review_doc)
    return "Review added successfully"

# Example usage:
result = add_review(stars=input("How many stars do you want to give? "), useful=input("How useful are they from 1-5? "), funny=input("How funny are they from 1-5? "), cool=input("How cool are they from 1-5? "), text=input("comment: "), business_name="Abby Rappoport, LAC, CMQ", user_name="John Doe")
print(result)


Review added successfully


In [23]:
# Access the collections
admin_collection = db['admin']
business_collection = db['business']

def email_exists(email):
    # Check if the email exists in the admin collection
    existing_admin = admin_collection.find_one({"admin_email": email})
    return existing_admin is not None

def username_exist(username):
    # Check if the username exists in the admin collection
    existing_admin = admin_collection.find_one({"admin_username": username})
    return existing_admin is not None

def business_exists(business):
    # Check if the email exists in the admin collection
    existing_admin = admin_collection.find_one({"business_id": business})
    return existing_admin is not None

def add_admin(business_name, admin_name, admin_username, admin_password, admin_email):
    if email_exists(admin_email):
        return "Email already exists for another admin"
    if username_exist(admin_username):
        return "Username already exists for another admin"
    if business_exists(business_id):
        return "Business already exists. One business can only have one admin"

    # Generate new random IDs for review and user
    admin_id = str(uuid.uuid4())

    # Find business_id based on business_name
    business = business_collection.find_one({"name": business_name})
    if not business:
        return "Business not found"
    business_id = business['business_id']

    # Construct the review document
    admin_doc = {
        "_id": admin_id,
        "admin_id": str(uuid.uuid4()),
        "admin_name": admin_name,
        "business_id": business_id,
        "admin_username": admin_username,
        "admin_password": admin_password,
        "admin_email": admin_email
    }

    # Insert the review document into the review collection
    admin_collection.insert_one(admin_doc)
    return "Admin data added successfully"

# Example usage:
result = add_admin(business_name=input("Business Name: "), admin_name=input("Admin Name: "), admin_username=input("Admin Username: "), admin_password=input("Password: "), admin_email=input("Email: "))
print(result)

Business not found


In [20]:
# AUFGABE 2
import pymongo


# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the database
db = client['yelp']  # Replace 'your_database' with your actual database name

# Access the collections
review_collection = db['review']
business_collection = db['business']

def notify_low_rating_reviews(business_name, threshold_percentage, last_n_reviews):
    # Get the business_id based on the business_name
    business = business_collection.find_one({"name": business_name})
    if not business:
        return "Business not found"

    business_id = business['business_id']

    # Find the last 'n' reviews for the given business_id
    recent_reviews = list(review_collection.find(
        {"business_id": business_id, "stars": {"$in": [1, 2]}}
    ).sort([("timestamp", -1)]).limit(last_n_reviews))


    # Calculate the percentage of reviews with stars 1 or 2
    low_rating_count = sum(1 for review in recent_reviews if review['stars'] in [2])
    total_reviews = len(recent_reviews)

    percentage = (low_rating_count / total_reviews) * 100 if total_reviews > 0 else 0

    # Notify if the threshold percentage is exceeded
    if percentage >= threshold_percentage:
        return f"Threshold exceeded: {percentage}% of the last {last_n_reviews} reviews for '{business_name}' have low ratings. {business} {recent_reviews} {low_rating_count} {total_reviews}"

    
    return f"{percentage}% of the last {last_n_reviews} reviews for '{business_name}' have low ratings. Threshold not exceeded. {business} {recent_reviews} {low_rating_count} {total_reviews}"


# Make the sign in/up feature
# Make the notification feature (in the website /& email)
# Make sure the function works correctly
# Use redis to make the notification feature

# Example usage:
threshold = 50  # Set the threshold percentage
last_reviews = 30  # Set the number of last reviews to consider
business_name = "Turning Point of North Wales"  # Replace with the business name you want to monitor

notification = notify_low_rating_reviews(business_name, threshold, last_reviews)
print(notification)





Threshold exceeded: 56.666666666666664% of the last 30 reviews for 'Turning Point of North Wales' have low ratings. {'_id': ObjectId('656b29078895f3b51cb6d3b3'), 'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw', 'name': 'Turning Point of North Wales', 'address': '1460 Bethlehem Pike', 'city': 'North Wales', 'state': 'PA', 'postal_code': '19454', 'latitude': 40.2101961875, 'longitude': -75.2236385919, 'stars': 3, 'review_count': 169, 'is_open': 1, 'attributes': {'NoiseLevel': "u'average'", 'HasTV': 'False', 'RestaurantsAttire': "'casual'", 'BikeParking': 'False', 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}", 'WiFi': "'free'", 'DogsAllowed': 'False', 'Alcohol': "'none'", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'WheelchairAccessible': 'True', 'BusinessAcceptsBitcoin': 'Fa